#Imports

In [ ]:
#@title
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
import numpy as np
!git clone https://github.com/bosemessi/StatsbombOpenData --q
!pip install mplsoccer --q
from mplsoccer import Pitch, VerticalPitch
import ipywidgets as widgets
from ipywidgets import interact,interactive
import matplotlib.pyplot as plt
import matplotlib as mpl 
import matplotlib.font_manager as fm
!pip install highlight-text --q
from highlight_text import ax_text, fig_text
!pip install adjustText --q
from adjustText import adjust_text
!pip install gdown --quiet
import gdown
import matplotlib.patheffects as path_effects
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
import mpl_toolkits.axisartist.angle_helper as angle_helper
from matplotlib.projections import PolarAxes
from mpl_toolkits.axisartist.grid_finder import (FixedLocator, MaxNLocator,
                                                 DictFormatter)
import matplotlib.patches as patches
from PIL import Image
from io import StringIO, BytesIO
from tqdm import tqdm
import requests 
import warnings
warnings.filterwarnings('ignore')
from matplotlib.colors import LinearSegmentedColormap
from scipy.interpolate import RegularGridInterpolator

fatal: destination path 'StatsbombOpenData' already exists and is not an empty directory.


#Add xT

In [ ]:
#@title
%%time
df = pd.read_parquet('/content/StatsbombOpenData/WC2018.parquet').reset_index(drop=True)
Df = df[df.location.notna()].reset_index(drop=True)
touchbasedactions = ['Pass', 'Carry', 'Dribble','Foul Won', 'Interception', 'Duel','50/50',
       'Ball Recovery', 'Dispossessed', 'Block', 'Clearance','Miscontrol', 'Goal Keeper', 'Shot']
Df['isTouch'] = np.where(Df.type_name.isin(touchbasedactions),1,0)
Df = Df[(Df.isTouch==1)].reset_index(drop=True)
Df[['x','y']] = np.array(list(Df.location))
Df['endloc'] = np.where(Df.type_name=='Pass',Df.pass_end_location,
                        np.where(Df.type_name=='Carry',Df.carry_end_location,Df.location))
Df[['endX','endY']] = np.array(list(Df.endloc))

binx = [10*i for i in range(13)]
biny = [10*i for i in range(9)]
for cols in ['x','endX']:
    s = pd.cut(Df[cols], bins=binx, include_lowest=True)
    Df['zone_'+cols] = pd.Series(data=pd.IntervalIndex(s).right, index = s.index)/10
for cols in ['y','endY']:
    s = pd.cut(Df[cols], bins=biny, include_lowest=True)
    Df['zone_'+cols] = pd.Series(data=pd.IntervalIndex(s).right, index = s.index)/10
Df['zone_start'] = Df['zone_x'] + (Df['zone_y']-1)*12
Df['zone_end'] = Df['zone_endX'] + (Df['zone_endY']-1)*12

xtd = np.array([[0.00638303, 0.00779616, 0.00844854, 0.00977659, 0.01126267,
        0.01248344, 0.01473596, 0.0174506 , 0.02122129, 0.02756312,
        0.03485072, 0.0379259 ],
       [0.00750072, 0.00878589, 0.00942382, 0.0105949 , 0.01214719,
        0.0138454 , 0.01611813, 0.01870347, 0.02401521, 0.02953272,
        0.04066992, 0.04647721],
       [0.0088799 , 0.00977745, 0.01001304, 0.01110462, 0.01269174,
        0.01429128, 0.01685596, 0.01935132, 0.0241224 , 0.02855202,
        0.05491138, 0.06442595],
       [0.00941056, 0.01082722, 0.01016549, 0.01132376, 0.01262646,
        0.01484598, 0.01689528, 0.0199707 , 0.02385149, 0.03511326,
        0.10805102, 0.25745362],
       [0.00941056, 0.01082722, 0.01016549, 0.01132376, 0.01262646,
        0.01484598, 0.01689528, 0.0199707 , 0.02385149, 0.03511326,
        0.10805102, 0.25745362],
       [0.0088799 , 0.00977745, 0.01001304, 0.01110462, 0.01269174,
        0.01429128, 0.01685596, 0.01935132, 0.0241224 , 0.02855202,
        0.05491138, 0.06442595],
       [0.00750072, 0.00878589, 0.00942382, 0.0105949 , 0.01214719,
        0.0138454 , 0.01611813, 0.01870347, 0.02401521, 0.02953272,
        0.04066992, 0.04647721],
       [0.00638303, 0.00779616, 0.00844854, 0.00977659, 0.01126267,
        0.01248344, 0.01473596, 0.0174506 , 0.02122129, 0.02756312,
        0.03485072, 0.0379259 ]]).flatten()
startXTDF = pd.DataFrame(data=xtd,columns=['xT_start'])
startXTDF['zone_start'] = [i+1 for i in range(96)]
endXTDF = pd.DataFrame(data=xtd,columns=['xT_end'])
endXTDF['zone_end'] = [i+1 for i in range(96)]

Df = Df.merge(startXTDF,on=['zone_start'],how='left')
Df = Df.merge(endXTDF,on=['zone_end'],how='left')
Df['xT'] = Df['xT_end'] - Df['xT_start']

CPU times: user 11 s, sys: 697 ms, total: 11.7 s
Wall time: 11.2 s


#Create xT Facilitated, then save

In [ ]:
Df['xT'] = np.where(Df.type_name.isin(['Pass','Carry']) & Df.pass_outcome_name.isna(),Df.xT,0)
Df['xT Facilitated'] = np.where(Df.team_name==Df.team_name.shift(-1),Df.xT.shift(-1).fillna(value=0),0)
groupedstats = Df.groupby('player_name').agg({'xT':'sum','xT Facilitated':'sum'}).reset_index()
groupedstats.rename(columns={"player_name": "name"}, errors="raise",inplace=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
groupedstats.to_parquet('/content/drive/MyDrive/Norwich/xT.parquet',index=False)